**title.basics.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) - the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) - the more popular title / the title used by the filmmakers on promotional materials at the point of release
- startYear (YYYY) - represents the release year of a title. In the case of TV Series, it is the series start year
- runtimeMinutes - primary runtime of the title, in minutes
- genres (string array) - includes up to three genres associated with the title

**title.crew.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- directors (array of nconsts) - director(s) of the given title
- writers (array of nconsts) - writer(s) of the given title

**title.ratings.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- averageRating - weighted average of all the individual user ratings

**name.basics.tsv.gz**
- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string) - name by which the person is most often credited

**title.principals.tsv.gz**
- tconst (string) - alphanumeric unique identifier of the title
- nconst (string) - alphanumeric unique identifier of the name/person
- category (string) - the category of job that person was in
- job (string) - the specific job title if applicable, else '\N'

# Setup

Pip installs

In [1]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


Python imports

In [135]:
import os.path as path
from pyspark.sql import SparkSession
from pyspark.sql import column
from pyspark.sql.functions import *
import matplotlib.pyplot as plt

In [136]:
title_basics_dataset = "title.basics.tsv"

title_crew_dataset = "title.crew.tsv"

title_ratings_dataset = "title.ratings.tsv"

name_basics_dataset = "name.basics.tsv"

title_principals_dataset = "title.principals.tsv"

movie_description_dataset1 = "movies_metadata.csv"

movie_description_dataset2 = "mpst_full_data.csv"


if not(path.isfile(title_basics_dataset)):
  !wget https://datasets.imdbws.com/title.basics.tsv.gz
  !gzip -d title.basics.tsv.gz

if not(path.isfile(title_crew_dataset)):
  !wget https://datasets.imdbws.com/title.crew.tsv.gz
  !gzip -d title.crew.tsv.gz

if not(path.isfile(title_ratings_dataset)):
  !wget https://datasets.imdbws.com/title.ratings.tsv.gz
  !gzip -d title.ratings.tsv.gz

if not(path.isfile(name_basics_dataset)):
  !wget https://datasets.imdbws.com/name.basics.tsv.gz
  !gzip -d name.basics.tsv.gz

if not(path.isfile(title_principals_dataset)):
  !wget https://datasets.imdbws.com/title.principals.tsv.gz
  !gzip -d title.principals.tsv.gz

if not(path.isfile(movie_description_dataset1)):
  !wget https://raw.githubusercontent.com/ZakaryLepine/SOEN471-Project-Datasets/main/movies_metadata.csv

if not(path.isfile(movie_description_dataset2)):
  !wget https://raw.githubusercontent.com/ZakaryLepine/SOEN471-Project-Datasets/main/mpst_full_data.csv

# Preparing Dataset

Helpers

In [137]:
def init_spark():
  return SparkSession \
        .builder \
        .appName("Spark") \
        .config("spark.executor.memory", "8g") \
        .config("spark.driver.memory", "8g") \
        .getOrCreate()

spark = init_spark()


Create base dataframes

In [138]:
spark

In [139]:
title_basics_df = spark.read.option("delimiter", "\t").option("header", True).csv(title_basics_dataset)

title_crew_df = spark.read.option("delimiter", "\t").option("header", True).csv(title_crew_dataset)

title_ratings_df = spark.read.option("delimiter", "\t").option("header", True).csv(title_ratings_dataset)

name_basics_df = spark.read.option("delimiter", "\t").option("header", True).csv(name_basics_dataset)

title_principals_df = spark.read.option("delimiter", "\t").option("header", True).csv(title_principals_dataset)

movie_description_df1 = spark.read.option("delimiter", ",").option("header", True).csv(movie_description_dataset1)

movie_description_df2 = spark.read.option("delimiter", ",").option("header", True).csv(movie_description_dataset2)

In [140]:
movie_description_df1.columns

['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [141]:
# movie_description_df2.count()

In [142]:
title_basics_df.show()
title_crew_df.show()
title_ratings_df.show()
name_basics_df.show()
title_principals_df.show()
movie_description_df1.show()
movie_description_df2.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

## Filters

Filter entries for only movie entries

In [143]:
title_basics_df = title_basics_df.filter(title_basics_df.titleType == "movie")
#title_basics_df.show()

Remove entries where the genres is null (\N)

In [144]:
title_basics_df = title_basics_df.filter(title_basics_df.genres != "\\N")
#title_basics_df.show()

## Integration

Separate genres and keep single genre

Cases:

| In 0 | In 1 | In 2 |
|------|------|------|
| T    | T    | T    |
| T    | T    | F    |
| T    | F    | T    |
| F    | T    | T    |
| T    | T    | F    |
| T    | F    | T    |
| F    | T    | T    |
| F    | F    | F    |

In [145]:
#When there are multiple genres:
#if only 1 from the list is in the genre column -> pick genre from list
#if multiple genres from the list is in the genre column -> weigh by count?
drama_entry_count = 0
comedy_entry_count = 0
horror_entry_count = 0
romance_entry_count = 0
action_entry_count = 0
thriller_entry_count = 0

genre_column = split(title_basics_df["genres"], ",")

genres_to_keep = ["Drama", "Comedy", "Horror", "Action"]
# genres_to_keep = ["Drama", "Comedy", "Horror", "Thriller", "Romance", "Action"]

test = title_basics_df.filter(genre_column.getItem(0).isin(genres_to_keep))
#test = title_basics_df.filter(genre_column.getItem(0).isin(genres_to_keep))
test = test.withColumn("genre", genre_column.getItem(0))

test2 = title_basics_df.filter((genre_column.getItem(1).isin(genres_to_keep)) & ~(genre_column.getItem(0).isin(genres_to_keep) ))
#test2 = title_basics_df.filter(genre_column.getItem(1).isin(genres_to_keep))
test2 = test2.withColumn("genre", genre_column.getItem(1))

test3 = title_basics_df.filter((genre_column.getItem(2).isin(genres_to_keep)) & ~(genre_column.getItem(0).isin(genres_to_keep) | genre_column.getItem(1).isin(genres_to_keep)))
#test3 = title_basics_df.filter(genre_column.getItem(2).isin(genres_to_keep))
test3 = test3.withColumn("genre", genre_column.getItem(2))

title_basics_df = test.union(test2).union(test3)
#title_basics_df.show()

'''
#title_basics_df.groupBy('genres').count().orderBy(desc("count")).show()

intermediate_df = title_basics_df.withColumn("genre1", genre_column.getItem(0))

genre_rows1 = intermediate_df.where((intermediate_df.genre1 == "Drama") | (intermediate_df.genre1 == "Comedy") | (intermediate_df.genre1 == "Horror") | (intermediate_df.genre1 == "Thriller") | (intermediate_df.genre1 == "Romance") | (intermediate_df.genre1 == "Action")).groupBy('genre1').count().orderBy("genre1").take(6)

intermediate_df = title_basics_df.withColumn("genre2", genre_column.getItem(1))

genre_rows2 = intermediate_df.where((intermediate_df.genre2 == "Drama") | (intermediate_df.genre2 == "Comedy") | (intermediate_df.genre2 == "Horror") | (intermediate_df.genre2 == "Thriller") | (intermediate_df.genre2 == "Romance") | (intermediate_df.genre2 == "Action")).groupBy('genre2').count().orderBy("genre2").take(6)

intermediate_df = title_basics_df.withColumn("genre3", genre_column.getItem(2))

genre_rows3 = intermediate_df.where((intermediate_df.genre3 == "Drama") | (intermediate_df.genre3 == "Comedy") | (intermediate_df.genre3 == "Horror") | (intermediate_df.genre3 == "Thriller") | (intermediate_df.genre3 == "Romance") | (intermediate_df.genre3 == "Action")).groupBy('genre3').count().orderBy("genre3").take(6)

for row in genre_rows1:
  if (row["genre1"] == "Action"):
    action_entry_count = action_entry_count + row["count"]

  if (row["genre1"] == "Comedy"):
    comedy_entry_count = comedy_entry_count + row["count"]

  if (row["genre1"] == "Drama"):
    drama_entry_count = drama_entry_count + row["count"]

  if (row["genre1"] == "Horror"):
    horror_entry_count = horror_entry_count + row["count"]

  if (row["genre1"] == "Romance"):
    romance_entry_count = romance_entry_count + row["count"]

  if (row["genre1"] == "Thriller"):
    thriller_entry_count = thriller_entry_count + row["count"]

for row in genre_rows2:
  if (row["genre2"] == "Action"):
    action_entry_count = action_entry_count + row["count"]

  if (row["genre2"] == "Comedy"):
    comedy_entry_count = comedy_entry_count + row["count"]

  if (row["genre2"] == "Drama"):
    drama_entry_count = drama_entry_count + row["count"]

  if (row["genre2"] == "Horror"):
    horror_entry_count = horror_entry_count + row["count"]

  if (row["genre2"] == "Romance"):
    romance_entry_count = romance_entry_count + row["count"]

  if (row["genre2"] == "Thriller"):
    thriller_entry_count = thriller_entry_count + row["count"]

for row in genre_rows3:
  if (row["genre3"] == "Action"):
    action_entry_count = action_entry_count + row["count"]

  if (row["genre3"] == "Comedy"):
    comedy_entry_count = comedy_entry_count + row["count"]

  if (row["genre3"] == "Drama"):
    drama_entry_count = drama_entry_count + row["count"]

  if (row["genre3"] == "Horror"):
    horror_entry_count = horror_entry_count + row["count"]

  if (row["genre3"] == "Romance"):
    romance_entry_count = romance_entry_count + row["count"]

  if (row["genre3"] == "Thriller"):
    thriller_entry_count = thriller_entry_count + row["count"]

genre_count_dic = {"Action": action_entry_count, "Comedy": comedy_entry_count, "Drama": drama_entry_count, "Horror": horror_entry_count, "Romance": romance_entry_count, "Thriller": thriller_entry_count}
sorted_genre_count_dic = dict(sorted(genre_count_dic.items(), key=lambda item: item[1], reverse=True))
print(sorted_genre_count_dic)

genre_column = split(title_basics_df["genres"], ",")

title_basics_df = title_basics_df.withColumn("genre", genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Drama") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Comedy") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Action") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Romance") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Thriller") == 0 else \
       genre_column.getItem(0) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Horror") == 0 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Drama") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Comedy") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Action") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Romance") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Thriller") == 1 else \
       genre_column.getItem(1) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Horror") == 1 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Drama") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Comedy") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Action") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Romance") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Thriller") == 2 else \
       genre_column.getItem(2) if [genre_column.getItem(0), genre_column.getItem(1), genre_column.getItem(2)].index("Horror") == 2 else None)
'''

'\n#title_basics_df.groupBy(\'genres\').count().orderBy(desc("count")).show()\n\nintermediate_df = title_basics_df.withColumn("genre1", genre_column.getItem(0))\n\ngenre_rows1 = intermediate_df.where((intermediate_df.genre1 == "Drama") | (intermediate_df.genre1 == "Comedy") | (intermediate_df.genre1 == "Horror") | (intermediate_df.genre1 == "Thriller") | (intermediate_df.genre1 == "Romance") | (intermediate_df.genre1 == "Action")).groupBy(\'genre1\').count().orderBy("genre1").take(6)\n\nintermediate_df = title_basics_df.withColumn("genre2", genre_column.getItem(1))\n\ngenre_rows2 = intermediate_df.where((intermediate_df.genre2 == "Drama") | (intermediate_df.genre2 == "Comedy") | (intermediate_df.genre2 == "Horror") | (intermediate_df.genre2 == "Thriller") | (intermediate_df.genre2 == "Romance") | (intermediate_df.genre2 == "Action")).groupBy(\'genre2\').count().orderBy("genre2").take(6)\n\nintermediate_df = title_basics_df.withColumn("genre3", genre_column.getItem(2))\n\ngenre_rows3 = 

In [146]:
title_crew_df.show()

+---------+-------------------+---------+
|   tconst|          directors|  writers|
+---------+-------------------+---------+
|tt0000001|          nm0005690|       \N|
|tt0000002|          nm0721526|       \N|
|tt0000003|          nm0721526|       \N|
|tt0000004|          nm0721526|       \N|
|tt0000005|          nm0005690|       \N|
|tt0000006|          nm0005690|       \N|
|tt0000007|nm0005690,nm0374658|       \N|
|tt0000008|          nm0005690|       \N|
|tt0000009|          nm0085156|nm0085156|
|tt0000010|          nm0525910|       \N|
|tt0000011|          nm0804434|       \N|
|tt0000012|nm0525908,nm0525910|       \N|
|tt0000013|          nm0525910|       \N|
|tt0000014|          nm0525910|       \N|
|tt0000015|          nm0721526|       \N|
|tt0000016|          nm0525910|       \N|
|tt0000017|nm1587194,nm0804434|       \N|
|tt0000018|          nm0804434|       \N|
|tt0000019|          nm0932055|       \N|
|tt0000020|          nm0010291|       \N|
+---------+-------------------+---

In [147]:
director_column = split(title_crew_df["directors"], ",")

Separate directors and keep single director

In [148]:
director_column = split(title_crew_df["directors"], ",")
title_crew_df = title_crew_df.withColumn("director", director_column.getItem(0))
#title_crew_df.show()
title_crew_df = title_crew_df.drop("directors")
#title_crew_df.show()

In [149]:
title_crew_df.show()

+---------+---------+---------+
|   tconst|  writers| director|
+---------+---------+---------+
|tt0000001|       \N|nm0005690|
|tt0000002|       \N|nm0721526|
|tt0000003|       \N|nm0721526|
|tt0000004|       \N|nm0721526|
|tt0000005|       \N|nm0005690|
|tt0000006|       \N|nm0005690|
|tt0000007|       \N|nm0005690|
|tt0000008|       \N|nm0005690|
|tt0000009|nm0085156|nm0085156|
|tt0000010|       \N|nm0525910|
|tt0000011|       \N|nm0804434|
|tt0000012|       \N|nm0525908|
|tt0000013|       \N|nm0525910|
|tt0000014|       \N|nm0525910|
|tt0000015|       \N|nm0721526|
|tt0000016|       \N|nm0525910|
|tt0000017|       \N|nm1587194|
|tt0000018|       \N|nm0804434|
|tt0000019|       \N|nm0932055|
|tt0000020|       \N|nm0010291|
+---------+---------+---------+
only showing top 20 rows



Separate writers and keep single writer

In [150]:
writer_column = split(title_crew_df["writers"], ",")
title_crew_df = title_crew_df.withColumn("writer", writer_column.getItem(0))
#title_crew_df.show()
title_crew_df = title_crew_df.drop("writers")
#title_crew_df.show()

Get director names

In [151]:
director_df = title_crew_df.select("director", "tconst")
renamed_director_df = director_df.withColumnRenamed("director", "nconst")

director_id_name_df = renamed_director_df.join(name_basics_df, on="nconst").select("tconst", "primaryName")
director_id_name_df = director_id_name_df.withColumnRenamed("primaryName", "directorPrimaryName")
#director_id_name_df.show()

Get writer names

In [152]:
writer_df = title_crew_df.select("writer", "tconst")
renamed_writer_df = writer_df.withColumnRenamed("writer", "nconst")

writer_id_name_df = renamed_writer_df.join(name_basics_df, on="nconst").select("tconst", "primaryName")
writer_id_name_df = writer_id_name_df.withColumnRenamed("primaryName", "writerPrimaryName")
#writer_id_name_df.show()

In [153]:
title_basics_and_title_crew_and_title_ratings_df = title_basics_df.join(title_ratings_df, on="tconst").join(director_id_name_df, on="tconst").join(writer_id_name_df, on="tconst")
#title_basics_and_title_crew_and_title_ratings_df.show()

In [154]:
title_basics_and_title_crew_and_title_ratings_df.columns

['tconst',
 'titleType',
 'primaryTitle',
 'originalTitle',
 'isAdult',
 'startYear',
 'endYear',
 'runtimeMinutes',
 'genres',
 'genre',
 'averageRating',
 'numVotes',
 'directorPrimaryName',
 'writerPrimaryName']

In [155]:
# title_basics_and_title_crew_and_title_ratings_df.count()

Get actor names

In [156]:
person_df = title_principals_df.select("tconst", "nconst", "category")
#person_df.show()

actor_df = person_df.filter(person_df.category == "actor")
#person_df = person_df.filter((person_df.category == "self") | (person_df.category == "cinematographer") | (person_df.category == "composer") | (person_df.category == "producer") | (person_df.category == "editor") | (person_df.category == "actor"))
#actor_df.show()

actor_df = actor_df.join(name_basics_df, on="nconst").select("tconst", "primaryName")
actor_id_name_df = actor_df.withColumnRenamed("primaryName", "actorPrimaryName")
#actor_id_name_df.show()
title_basics_and_title_crew_and_title_ratings_actor_df = actor_id_name_df.groupBy("tconst").agg(collect_list('actorPrimaryName').alias("actors"))
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.withColumn("actor1", title_basics_and_title_crew_and_title_ratings_actor_df.actors[0])
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.withColumn("actor2", title_basics_and_title_crew_and_title_ratings_actor_df.actors[1])
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.withColumn("actor3", title_basics_and_title_crew_and_title_ratings_actor_df.actors[2])
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.withColumn("actor4", title_basics_and_title_crew_and_title_ratings_actor_df.actors[3])
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_df.join(title_basics_and_title_crew_and_title_ratings_actor_df, on="tconst")
#.groupBy("actorPrimaryName").pivot("tconst").agg(first("actorPrimaryName"))
#(nth_value("actorPrimaryName", 0).alias("actor1"), nth_value("actorPrimaryName", 1).alias("actor2"), nth_value("actorPrimaryName", 2).alias("actor3"), nth_value("actorPrimaryName", 3).alias("actor4"))
#test.show()
#title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_df.join(actor_id_name_df, on="tconst")
#title_basics_and_title_crew_and_title_ratings_actor_df.show()

In [157]:
# title_basics_and_title_crew_and_title_ratings_actor_df.count()

In [158]:
movie_description_df2.columns

['imdb_id', 'title', 'plot_synopsis', 'tags', 'split', 'synopsis_source']

In [159]:
title_basics_and_title_crew_and_title_ratings_actor_df.columns

['tconst',
 'titleType',
 'primaryTitle',
 'originalTitle',
 'isAdult',
 'startYear',
 'endYear',
 'runtimeMinutes',
 'genres',
 'genre',
 'averageRating',
 'numVotes',
 'directorPrimaryName',
 'writerPrimaryName',
 'actors',
 'actor1',
 'actor2',
 'actor3',
 'actor4']

In [160]:
movie_description_df2.head()

Row(imdb_id='tt0057603', title='I tre volti della paura', plot_synopsis='"Note: this synopsis is for the orginal Italian release with the segments in this certain order.Boris Karloff introduces three horror tales of the macabre and the supernatural known as the \'Three Faces of Fear\'.THE TELEPHONERosy (Michele Mercier) is an attractive, high-priced Parisian call-girl who returns to her spacious, basement apartment after an evening out when she immediately gets beset by a series of strange phone calls. The caller soon identified himself as Frank, her ex-pimp who has recently escaped from prison. Rosy is terrified for it was her testimony that landed the man in jail. Looking for solace, Rosy phones her lesbian lover Mary (Lynda Alfonsi). The two women have been estranged for some time, but Rosy is certain that she is the only one who can help her. Mary agrees to come over that night. Seconds later, Frank calls again, promising that no matter who she calls for protection, he will have hi

Get movie description

In [121]:
movie_description_df2 = movie_description_df2.withColumnRenamed("imdb_id", "tconst")
movie_description_df2=movie_description_df2.drop('title')
# movie_description_df1 = movie_description_df1.filter(movie_description_df1.overview.isNotNull())
# movie_description_df1 = movie_description_df1.withColumnRenamed("original_title", "primaryTitle")
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.join(movie_description_df2, on="tconst")

In [161]:
# movie_description_df2 = movie_description_df2.withColumnRenamed("imdb_id", "tconst")
# movie_description_df2=movie_description_df2.drop('title')
movie_description_df1 = movie_description_df1.filter(movie_description_df1.overview.isNotNull())
movie_description_df1 = movie_description_df1.withColumnRenamed("original_title", "primaryTitle")
title_basics_and_title_crew_and_title_ratings_actor_df = title_basics_and_title_crew_and_title_ratings_actor_df.join(movie_description_df1, on="primaryTitle")

In [162]:
title_basics_and_title_crew_and_title_ratings_actor_df.columns

['primaryTitle',
 'tconst',
 'titleType',
 'originalTitle',
 'isAdult',
 'startYear',
 'endYear',
 'runtimeMinutes',
 'genres',
 'genre',
 'averageRating',
 'numVotes',
 'directorPrimaryName',
 'writerPrimaryName',
 'actors',
 'actor1',
 'actor2',
 'actor3',
 'actor4',
 'adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [163]:
title_basics_and_title_crew_and_title_ratings_actor_df.count()

41311

In [164]:
def print_class_distribution(df):
    # Group the DataFrame by the "genre" column and count the occurrences of each genre
    genre_counts = df.groupBy("genre").count()
    
    # Show the genre counts
    genre_counts.show()
print_class_distribution(title_basics_and_title_crew_and_title_ratings_actor_df)

+------+-----+
| genre|count|
+------+-----+
| Drama|19906|
|Horror| 3360|
|Comedy|11629|
|Action| 6416|
+------+-----+



In [125]:
final_df = title_basics_and_title_crew_and_title_ratings_actor_df.drop('titleType', 'originalTitle', 'isAdult', 'endYear', 'genres', 'numVotes', 'tconst', "actors")
final_df = final_df.withColumnRenamed('primaryTitle', 'title')
final_df = final_df.withColumnRenamed('startYear', 'year')
final_df = final_df.withColumnRenamed('runtimeMinutes', 'runtime')
final_df = final_df.withColumnRenamed('averageRating', 'rating')
final_df = final_df.withColumnRenamed('directorPrimaryName', 'director')
final_df = final_df.withColumnRenamed('writerPrimaryName', 'writer')
final_df = final_df.withColumnRenamed('plot_synopsis', 'plot')

# final_df.show()
# final_df.persist()
# final_df.repartition(1).write.option("header", "true").save("mydata.csv")


In [126]:
final_df.columns

['title',
 'year',
 'runtime',
 'genre',
 'rating',
 'director',
 'writer',
 'actor1',
 'actor2',
 'actor3',
 'actor4',
 'plot',
 'tags',
 'split',
 'synopsis_source']

In [127]:
# movies releaed after 2021 to reduce data
# final_df=final_df.filter(final_df.year.cast("int") > 2021)

In [128]:
# Remove rows with null actor1 to actor4
final_df = final_df.filter(final_df.actor1.isNotNull())
final_df = final_df.filter(final_df.actor2.isNotNull())
final_df = final_df.filter(final_df.actor3.isNotNull())
final_df = final_df.filter(final_df.actor4.isNotNull())

In [129]:
title_basics_and_title_crew_and_title_ratings_actor_df

DataFrame[tconst: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: string, startYear: string, endYear: string, runtimeMinutes: string, genres: string, genre: string, averageRating: string, numVotes: string, directorPrimaryName: string, writerPrimaryName: string, actors: array<string>, actor1: string, actor2: string, actor3: string, actor4: string, plot_synopsis: string, tags: string, split: string, synopsis_source: string]

In [130]:
final_df.columns

['title',
 'year',
 'runtime',
 'genre',
 'rating',
 'director',
 'writer',
 'actor1',
 'actor2',
 'actor3',
 'actor4',
 'plot',
 'tags',
 'split',
 'synopsis_source']

In [131]:
final_df.head()

ERROR:root:KeyboardInterrupt while sending command.][Stage 747:> (0 + 0) / 22]
Traceback (most recent call last):
  File "/home/umang/.local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/umang/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
final_df.count()

In [ ]:
# print_class_distribution(

In [ ]:
# final_df_sample = final_df.sample(fraction=0.1, seed=42)  # Sample 10% of the data
# final_df_sample.show(2)

In [134]:
final_df.write.option("header", "true").csv("final_dataset_with_df2.csv")